In [1]:
import pandas as pd
from glob import glob
from os.path import join
import os
from collections import defaultdict
import ntpath
import numpy as np

In [2]:
def write_flow_dist(flow_dist):
    with open(join(dataroot,'flow_dist.csv'),'w') as f:
        for key in sorted(flow_dist.keys()):
            f.write('{},{}\n'.format(key,flow_dist[key]))
            

def calc_n_write_flow_observation(flow_dist,dataroot):
    #####################
    root = '/data/juma/data/ids18'
    gt_df = pd.read_csv(join(root,'CSVs/WS_l/flow_dist.csv'),encoding='utf-8',usecols=['Label','Count'],dtype={'Label':str,'Count':int})
    benign_list = pd.read_csv(join(root,'benign_list.csv'),header=None)[0].values
    short_attack_list = pd.read_csv(join(root,'short_attack_list.csv'),header=None)[0].values
    long_attack_list = pd.read_csv(join(root,'long_attack_list.csv'),header=None)[0].values
    ##########################

    #unique_labels,label_counts = np.unique(flowlabels,return_counts=True)
    observation_df = pd.DataFrame(columns=['Label','Count','Observation rate'])
    macro_avg = 0
    # benign
    rates = 0
    counts = 0
    for label in benign_list:
        if label in flow_dist.keys():
            gt_count = gt_df[gt_df['Label']==label]['Count'].values[0]
            count = flow_dist[label]
            rate = round(100*count/gt_count,2)
        else:
            count = 0
            rate = 0
        observation_df = observation_df.append({'Label':label,'Count':count,'Observation rate':rate},ignore_index=True)

    observation_rate_sum = 0
    count_sum = 0
    # short attacks
    for label in short_attack_list:
        if label in flow_dist.keys():
            gt_count = gt_df[gt_df['Label']==label]['Count'].values[0]
            count = flow_dist[label]
            rate = round(100*count/gt_count,2)
        else:
            count = 0
            rate = 0
        observation_rate_sum+=rate
        count_sum +=count
        observation_df = observation_df.append({'Label':label,'Count':count,'Observation rate':rate},ignore_index=True)

    # long attacks
    for label in long_attack_list:
        if label in flow_dist.keys():
            gt_count = gt_df[gt_df['Label']==label]['Count'].values[0]
            count = flow_dist[label]
            rate = round(100*count/gt_count,2)
        else:
            count = 0
            rate = 0
        observation_df = observation_df.append({'Label':label,'Count':count,'Observation rate':rate},ignore_index=True)
        observation_rate_sum+=rate
        count_sum +=count

    m_observation_rate = observation_rate_sum/(len(short_attack_list)+len(long_attack_list))
    observation_df = observation_df.append({'Label':'Macro average Observation Rate','Count':count_sum,'Observation rate':m_observation_rate},ignore_index=True)
    observation_df.to_csv(join(dataroot,'observation_rate.csv'),index=False,encoding='utf-8-sig')


In [3]:
def process_datadir(dataroot):
    print(dataroot)
    flow_dist = defaultdict(lambda: 0)
    for fn in glob(join(dataroot,'*Meter.csv')):
        print(ntpath.basename(fn))
        df = pd.read_csv(fn,usecols=['Flow ID','Label'],dtype={'Flow ID':str,'Label':str})
        flow_counts = df.groupby(['Label'],as_index=False).agg({'Flow ID':'nunique'})
        for row in flow_counts.iterrows():
            label = row[1]['Label']
            count =row[1]['Flow ID']
            flow_dist[label]+= count
    calc_n_write_flow_observation(flow_dist, dataroot)

In [5]:
from multiprocessing import Process 
roots = [
    '/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/SFS_SI_9.77_l',
    '/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/FFS_(8,16,4)_l',
    '/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/SGS_e_0.00285_l',
]
procs = [ Process(target=process_datadir, args=[root]) for root in roots]
for p in procs: p.start()
for p in procs: p.join()

/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/SFS_SI_9.77_l
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/FFS_(8,16,4)_l
/data/juma/data/ids18/sampler_memory_archive/CSVs_r_1.0_m_0.05/SR_10/SGS_e_0.00285_l
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv


Process Process-1:
Process Process-2:


KeyboardInterrupt: 

Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):


In [4]:
process_datadir('/data/juma/data/ids18/CSVs_r_1.0_m_0.05/SR_10/SFS_SI_9.77_l')

/data/juma/data/ids18/CSVs_r_1.0_m_0.05/SR_10/SFS_SI_9.77_l


FileNotFoundError: [Errno 2] No such file or directory: '/data/juma/data/ids18/CSVs_r_1.0_m_0.05/SR_10/SFS_SI_9.77_l/observation_rate.csv'